In [1]:
import pandas as pd

In [2]:
from ortools.sat.python import cp_model

In [3]:
def solve_it(input_data, time_limit, file_name='LOG'):
    # This one uses color_number as variable
    
    # parse the input
    lines = input_data.split('\n')
    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))

    model = cp_model.CpModel()
    color_number = model.NewIntVar(0, node_count, 'color_number')
    color = [
        model.NewIntVar(0, node_count, 'color%i' % i) for i in range(node_count)
    ]
    for i in range(node_count):
        model.Add(color[int(i)] <= color_number)
    for i, j in edges:
        model.Add(color[int(i)]!=color[int(j)])
    model.Minimize(color_number)
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    status = solver.Solve(model)
        
    solution = []
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(file_name+': OPT SOLUTION FOUND ON ' + str(solver.Value(color_number) + 1) + ' COLORS')
    for i in color:
        solution.append(solver.Value(i))

    return solution

In [4]:
def solve_it_for_one(input_data, file_name='LOG', color_number=-1):
    # This one uses color_number as constant
    
    # parse the input
    lines = input_data.split('\n')
    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))

    model = cp_model.CpModel()
    color = [
        model.NewIntVar(0, color_number-1, 'color%i' % i) for i in range(node_count)
    ]
    for i, j in edges:
        model.Add(color[int(i)]!=color[int(j)])
    model.Minimize(color_number)
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
        
    solution = []
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(file_name+': OPT SOLUTION FOUND ON ' + str(color_number) + ' COLORS')
    for i in color:
        solution.append(solver.Value(i))

    return solution

In [5]:
def get_result(file_name, time_limit=120, color_number=-1):
    with open('data/'+file_name, 'r') as input_data_file:
        input_data = input_data_file.read()
    if color_number == -1:
        solution = solve_it(input_data, time_limit, file_name)
    else:
        solution = solve_it_for_one(input_data, file_name, color_number)
    res = pd.DataFrame(solution, columns=['Color'])
    res.to_excel(file_name+'_res.xlsx', index=False, header=True)

In [6]:
for file_name in ['1_gc_50_3', '2_gc_70_7', '3_gc_100_5', '4_gc_250_9', '5_gc_500_1', '6_gc_1000_5']:
    get_result(file_name)

1_gc_50_3: OPT SOLUTION FOUND ON 6 COLORS


In [6]:
get_result('6_gc_1000_5', time_limit=60*30)

6_gc_1000_5: OPT SOLUTION FOUND ON 123 COLORS


In [6]:
get_result('1_gc_50_3', color_number=6)

1_gc_50_3: OPT SOLUTION FOUND ON 6 COLORS


In [ ]:
get_result('4_gc_250_9', color_number=78)
get_result('6_gc_1000_5', color_number=100)